This notebook contains my solutions to the first **12** Project Euler problems.

In [3]:
import time
import itertools
import functools
import math

## Problem 1

**Statement**

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.

In [53]:
start_time = time.time()
print(sum(filter(lambda n: (n % 3 == 0 or n % 5 == 0), range(1, 1000))))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

233168
Executed in <0.001 seconds.


## Problem 2

**Statement**

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

In [52]:
def fib():
    a = 0
    b = 1
    yield a
    yield b
    while True:
        c = a + b
        a = b
        b = c
        yield c

start_time = time.time()
print(sum([n for n in itertools.takewhile(lambda n: n < 4.0*10**6, fib()) if n % 2 == 0]))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

4613732
Executed in <0.001 seconds.


## Problem 3

**Statement**

What is the largest prime factor of the number 600851475143?

**Analysis**

The brute force solution is far too slow, instead we use a [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes). Note the easily verified fact that every potential prime factor $f_a$ of $n$ such that $f \geq \sqrt n$ necessarily has a cofactor $b$ such that $f_b \leq \sqrt n$, so we may stop our sieve there and then check the primality of cofactors to find the largest prime.

List indices begin at 0, not 1. To compensate I elected to add an extra value and then sheer off the first value from the boolean field.

In [455]:
def primes_less_than(mx):
    n = 2
    # Add an extra value to the field to count 0...mx.
    bool_field = [True]*(mx + 1)
    for num in range(2, mx + 1):
        if bool_field[num] == True:
            for mult in range(2 * num, mx, num):
                bool_field[mult] = False
    # Remove 0 and 1 from the results.
    return [n for n in range(len(bool_field)) if bool_field[n] == True][2:]

def is_prime(n):
    cp = primes_less_than(int(math.floor(math.sqrt(n))))
    return True not in map(lambda p: n % p == 0, cp)

def largest_prime_factor(n):
    cp = primes_less_than(int(math.floor(math.sqrt(n))))
    f_a = [p for p in cp if n % p == 0]
    f_b = [int(n / p) for p in f_a if is_prime(int(n / p))]
    if not cp or not f_a:
        return n
    return max(f_a + f_b)

start_time = time.time()
print(largest_prime_factor(600851475143))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

6857
Executed in <0.580 seconds.


## Problem 4

**Statement**

A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99. Find the largest palindrome made from the product of two 3-digit numbers.

In [462]:
def is_palindrome(n):
    return str(n) == str(n)[::-1]

start_time = time.time()
candidates = []
for i in range(600, 999):
    for j in range(600, 999):
        if is_palindrome(i * j):
            candidates.append(i * j)
print(max(candidates))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

906609
Executed in <0.240 seconds.


## Problem 5

**Statement**

2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder. What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

**Analysis**

This ought to be rephrased as: "what is the [least common multiple](https://en.wikipedia.org/wiki/Least_common_multiple) of 1, ... , 20?" A basic result from number theory is that such an LCM will be the maximum of the powers of the prime factorizations of the numbers.

Since we are working with small numbers we will use a "dirty" unoptimized prime factorization algorithm.

In [5]:
def factorize(n):
    for i in range(2, math.floor(n / 2)):
        if n % i == 0:
            return [i] + factorize(n / i)
    return [math.floor(n)]

def merge(l1, l2):
    ret = []
    for f in (set(l1) | set(l2)):
        ret += [f for numtimes in range(0, max(l1.count(f), l2.count(f)))]
    return ret

start_time = time.time()
solution_factorized = []
for f in range(20,1,-1):
    solution_factorized = merge(solution_factorized, factorize(f))
print(functools.reduce(lambda x, y: x * y, solution_factorized))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

232792560
Executed in <0.001 seconds.


## Problem 6

**Statement**

The sum of the squares of the first ten natural numbers is $12 + 22 + ... + 102 = 385$. The square of the sum of the first ten natural numbers is $(1 + 2 + ... + 10)2 = 552 = 3025$. Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 − 385 = 2640$.

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

In [519]:
start_time = time.time()
print(abs(sum([n**2 for n in range(0, 100)]) - sum(range(0, 100))**2))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

24174150
Executed in <0.000 seconds.


## Problem 7

**Statement**

By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13. What is the 10,001st prime number?

**Analysis**

The brute force way (first solution) is to count up using the `is_prime()` method we defined earlier, in problem 3.

The smarter way is to use the [prime number theorem](https://en.wikipedia.org/wiki/Prime_number_theorem): this theorem states that the prime numbers are asymptotically distributed, in particular with $p_n \sim n\log{n}$. So we build a sieve that's big enough to capture this number and run that instead!

Math, you crazy.

In [526]:
start_time = time.time()
i = n = 0
while n < 10001:
    if is_prime(i):
        n += 1
        i += 1
    else:
        i += 1
print(i - 1)
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

104723
Executed in <11.529 seconds.


In [533]:
start_time = time.time()
print(primes_less_than(int(15000*math.log(10001)))[9998])
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

104723
Executed in <0.081 seconds.


## Problem 8

**Statement**

Find the 13 adjacent digits in a crazy numeric sequence which have the highest total product.

In [573]:
n = '7316717653133062491922511967442657474235534919493496983520312774506326239578318016984801869478851843858615607891129494954595017379583319528532088055111254069874715852386305071569329096329522744304355766896648950445244523161731856403098711121722383113622298934233803081353362766142828064444866452387493035890729629049156044077239071381051585930796086670172427121883998797908792274921901699720888093776657273330010533678812202354218097512545405947522435258490771167055601360483958644670632441572215539753697817977846174064955149290862569321978468622482839722413756570560574902614079729686524145351004748216637048440319989000889524345065854122758866688116427171479924442928230863465674813919123162824586178664583591245665294765456828489128831426076900422421902267105562632111110937054421750694165896040807198403850962455444362981230987879927244284909188845801561660979191338754992005240636899125607176060588611646710940507754100225698315520005593572972571636269561882670428252483600823257530420752963450'

def prod_digits(num):
    s = 1
    while num:
        s *= (num % 10)
        num /= 10
        num = int(num)
    return s

start_time = time.time()
nums = [int(n[i:i + 13]) for i in range(0, len(n) - 13)]
products = [prod_digits(n) for n in nums]
print(max(products))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

23514624000
Executed in <0.009 seconds.


## Problem 9

**Statement**

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$. Find the product $abc$.

**Analysis**

Through substitution we know that $a + b + \sqrt{a^2 + b^2} = 1000$. We can find our answer by taking only values of a and b for which the latter value, $\sqrt{a^2 + b^2}$, is a whole number, and then flinging them against 1000.

There is a general property that all $c = 4n + 1$ for some $n$. So we can generate $c$ with which to test $a$ and $b$. It can be shown algebraically that, using the fact that the minimum value that any of the variables can take on is 1, the largest values they can take on are capped:

* $c_{max} = 998$, implying that $n_{max} = 249$.
* b_max given some n_max is demonstrably 998-4n.

We iterate through this constrained range to create our $a$, then test it against the Pythagorean relationship. The first $a$ that proves sufficient is our answer.

In [572]:
start_time = time.time()
for n in range(1,250):
    for b in range(1, 998 - 4*n):
        c = 4*n + 1
        a = 1000 - b - c
        if a**2 + b**2 == c**2:
            print(a*b*c)
            break
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

31875000
Executed in <0.267 seconds.


## Problem 10

**Statement**

Find the sum of all primes below 2 million.

**Analysis**

Solved this back in question number 3!

In [575]:
start_time = time.time()
print(sum(primes_less_than(2*10**6)))
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

142915828922
Executed in <1.356 seconds.


## Problem 11

Actually I skipped this. It's a rehash of problem 8, except more annoying.


## Problem 12

**Statement**

The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be $1 + 2 + 3 + 4 + 5 + 6 + 7 = 28$. The first ten terms would be:

$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, \ldots$

Let us list the factors of the first seven triangle numbers:

     1: 1
     3: 1,3
     6: 1,2,3,6
    10: 1,2,5,10
    15: 1,3,5,15
    21: 1,3,7,21
    28: 1,2,4,7,14,28

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?

**Analysis**

Per the [divisor function](https://en.wikipedia.org/wiki/Divisor_function), a result from number theory states that the number of divisors of some number $n$ with the prime factorization $\prod_{i=1}^n p_i^{k_i}$ is given by:

$$\tau (n) = (k_1 + 1)(k_2 + 1)(...)(k_n + 1)$$

For example:

$$4200 = 2^33^15^27^1 \rightarrow \tau (4200) = (3+1)(1+1)(2+1)(1+1) = 48$$

This gives us a fast way of counting the number of divisors of a triangle number.

In [7]:
def count_divisors(num):
    f = factorize(num)
    return functools.reduce(lambda x, y: x * y, ([f.count(n) + 1 for n in set(f)]))

def triangle_numbers():
    n = 5
    while True:
        yield sum(range(1,n))
        n += 1

start_time = time.time()
for t_n in triangle_numbers():
   if count_divisors(t_n) > 500:
        print(t_n)
        break
print("Executed in <%s seconds." % format((time.time() - start_time), '.3f'))

76576500
Executed in <7.350 seconds.
